In [1]:
#Import libraries
import os
import pandas as pd
import geopandas as gp
from skimage import io
import matplotlib.pyplot as plt
from osgeo import gdal
import json
import csv
import earthaccess


In [2]:
#Login
earthaccess.login(persist=True)

In [3]:
#Set time and location

field = gp.read_file('./Isla_Vista_Kelp.geojson')
bbox = tuple(list(field.total_bounds))
bbox #Display coordinate bounds
with open('./Isla_Vista_Kelp.geojson', 'r') as f:
    data = json.load(f)
# Extract the name
location = (data['name']).replace('.kmz', '').replace(' ', '_')
temporal = ("2018-04-01T00:00:00", "2018-07-01T00:00:00") #
location
dem_name = 'dem.tif'

In [4]:
#Search for satellite data from  Landsat 30m and Sentinel 30m
results = earthaccess.search_data(
    short_name=['HLSL30','HLSS30'],
    bounding_box=bbox,
    temporal=temporal,
    # cloud_cover=.5,
    count=-1
)

dem_results = earthaccess.search_data(
    short_name="ASTGTM",
    bounding_box=bbox)

print(results[0])
print(dem_results[0])

Granules found: 43
Granules found: 1
Collection: {'EntryTitle': 'HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m v2.0'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -120.25617766, 'Latitude': 34.20966703}, {'Longitude': -119.06537353, 'Latitude': 34.23552162}, {'Longitude': -119.09014717, 'Latitude': 35.2250277}, {'Longitude': -120.29518354, 'Latitude': 35.19820772}, {'Longitude': -120.25617766, 'Latitude': 34.20966703}]}}]}}}
Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2018-04-02T18:50:59.370Z', 'EndingDateTime': '2018-04-02T18:50:59.370Z'}}
Size(MB): 222.56018352508545
Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T11SKU.2018092T183921.v2.0/HLS.S30.T11SKU.2018092T183921.v2.0.SAA.tif', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T11SKU.2018092T183921.v2.0/HLS.S30.T11SKU.2018092T183921.v2.0.B05

In [5]:
folder_path = os.path.join(os.getcwd(),(f'imagery/{location}'))
if not os.path.isfile(os.path.join(folder_path, dem_name)):
        dem_path = earthaccess.download(dem_results[0], local_path=folder_path)
        os.rename(dem_path[0], os.path.join(folder_path,'dem.tif'))
        os.rename(dem_path[1], os.path.join(folder_path, 'num.tif'))
if not os.path.isdir(folder_path):
    os.mkdir(folder_path)
for i, granule in enumerate(results):
    if i > 15:
        break
    json_str = json.dumps(granule.__dict__)
    metadata = json.loads(json_str) 
    meta = metadata['render_dict']['meta']
    name = meta['native-id']

    #For some reason, attributes are parsed into a list in the HLS metadata. This reformats it into a dictionary.
    attributes_list = metadata['render_dict']['umm']['AdditionalAttributes']

    attributes = {attr['Name']: attr['Values'][0] for attr in attributes_list}
    print(attributes['MGRS_TILE_ID'])
    if not attributes['MGRS_TILE_ID'] == '11SKU':
        pass

    if(int(attributes['CLOUD_COVERAGE']) > 50): #Reject granules with large cloud cover, for now
        pass
    time = attributes['SENSING_TIME']
    tile_folder = attributes['MGRS_TILE_ID']
    tile_path = os.path.join(folder_path,tile_folder)
    if not os.path.isdir(tile_path):
         os.mkdir(tile_path)
    folder_name = (f'{name}')
    file_path = os.path.join(tile_path,folder_name)
    if not os.path.isdir(file_path):
        os.mkdir(file_path) #Make folder for granule 

    downloadPath = earthaccess.download(granule, local_path=file_path)
    csv_file = os.path.join(file_path, (f'{folder_name}_metadata.csv'))
    metadata_full_dict = {**attributes, **meta}
    metadata_full_dict['data_vis_url'] = granule.dataviz_links()

    with open(csv_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(metadata_full_dict.keys())
        writer.writerow(metadata_full_dict.values())




 Getting 1 granules, approx download size: 0.02 GB


QUEUEING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/2 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/2 [00:00<?, ?it/s]

11SKU
 Getting 1 granules, approx download size: 0.22 GB


...

COLLECTING RESULTS | :   0%|          | 0/15 [00:00<?, ?it/s]

10SGD
 Getting 1 granules, approx download size: 0.21 GB


QUEUEING TASKS | :   0%|          | 0/18 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/18 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/18 [00:00<?, ?it/s]